In [3]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
#globals
TOTAL_FEATURES = 8
BACK_CANDLES = 10
FUTURE_DAYS = 1

def create_stock_dictionary():
    dictionary = {}
    current_directory = os.getcwd()
    folder_name = 'S&P500'
    folder_path = os.path.join(current_directory, folder_name)

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            name = file_name.split("_")[0]
            file_path = os.path.join(folder_path, file_name)
            dictionary[name] = pd.read_csv(file_path)
            
    return dictionary


In [6]:
import random
stock_data = create_stock_dictionary()
random_keys = random.sample(list(stock_data.keys()), 5)
random_keys


['ELV', 'MSCI', 'CI', 'DRI', 'ALGN']

In [ ]:
n_days = BACK_CANDLES
m_days = FUTURE_DAYS
random_stock_data = {}
for i in random_keys:
    random_stock_data[i] = stock_data[i]

In [28]:
stacked_stock_data = pd.concat((random_stock_data.values()), ignore_index=True, axis=0)
stacked_stock_data["TargetNextClose"] = stacked_stock_data["Adj Close"].shift(-1)
stacked_stock_data.drop(['Close'], axis=1, inplace=True)
stacked_stock_data.rename(columns = {'Datetime':'Date'}, inplace = True) 
scaler = MinMaxScaler()
for i in range(1, stacked_stock_data.shape[1]):
    column_data = stacked_stock_data.iloc[:, i].values.reshape(-1, 1)
    stacked_stock_data.iloc[:, i] = scaler.fit_transform(column_data)
stacked_stock_data = stacked_stock_data.dropna()
stacked_stock_data


C:\Users\Rober\AppData\Local\Temp\ipykernel_19548\4047666461.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.03466707 0.02392899 0.01355521 ... 0.00757541 0.00596104 0.03030032]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  stacked_stock_data.iloc[:, i] = scaler.fit_transform(column_data)


,Date,Open,High,Low,Adj Close,Volume,TargetNextClose
0,2022-06-28 09:30:00-04:00,0.753368,0.749387,0.744343,0.748650,0.034667,0.742822
1,2022-06-28 10:30:00-04:00,0.748751,0.742680,0.745809,0.742822,0.023929,0.732655
2,2022-06-28 11:30:00-04:00,0.743255,0.735746,0.735630,0.732655,0.013555,0.724957
3,2022-06-28 12:30:00-04:00,0.730743,0.725003,0.728302,0.724957,0.010410,0.722698
4,2022-06-28 13:30:00-04:00,0.725347,0.719206,0.725833,0.722698,0.010243,0.722828
...,...,...,...,...,...,...,...
16328,2024-03-06 10:30:00-05:00,0.382180,0.380483,0.385151,0.384012,0.005178,0.385572
16329,2024-03-06 11:30:00-05:00,0.383439,0.381830,0.385493,0.385572,0.003412,0.387581
16330,2024-03-06 12:30:00-05:00,0.385358,0.383433,0.388072,0.387581,0.005675,0.383932
16331,2024-03-06 13:30:00-05:00,0.388146,0.383522,0.384951,0.383932,0.007575,0.385012


In [44]:
xs, ys = [], [] # lists for our features/labels for LSTM

for i in range(len(stacked_stock_data) - n_days): 

    v = stacked_stock_data.iloc[i:(i + n_days), :]

    if v['Date'].iloc[-1] <= v['Date'].iloc[0]: 
        continue

    xs.append(v.iloc[:, :-1].to_numpy())
    ys.append(stacked_stock_data.iloc[i + n_days, len(stacked_stock_data.columns) - 1])
xs = np.array(xs)
ys = np.array(ys)


In [45]:
# Define some type of LSTM model
from keras.layers import LSTM
from keras.layers import Dense
import tensorflow as tf
from keras import Sequential
from keras import optimizers
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Dense, GRU

model = Sequential()
model.add(GRU(11, input_shape=(xs.shape[1], xs.shape[2])))
model.add(Dense(11, activation="relu"))
model.add(Dense(1))
model.compile(loss='mae', optimizer=Adam(1 / 1000))
print(model.summary())

c:\Users\Rober\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 11)             │           627 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 11)             │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771 (3.01 KB)

 Trainable params: 771 (3.01 KB)

 Non-trainable params: 0 (0.00 B)

None
